In [1]:
from IPython.display import clear_output

!git clone https://github.com/Katerina5649/clrs

%cd clrs
!pip install -r requirements/requirements.txt
#!pip install dm-clrs
#for assert
!pip install chex

clear_output(wait=False)

In [2]:
import clrs
import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
import importlib
importlib.reload(clrs)
from jax import random
from jax import grad

import haiku as hk
import jax
import jax.numpy as jnp
import optax
import numpy as np
import pandas as pd
import plotnine as gg

batch_size = 1
hidden_dim = 3
nb_nodes = 3


def loss_fn(data):
  mlp = hk.Sequential([
      clrs._src.processors.GATv2(10, 1)
  ])


  logits = mlp(data[0], data[1], data[2], data[3], data[4])
  return jnp.mean(logits)

loss_fn_t = hk.transform(loss_fn)

node_fts = random.uniform(random.PRNGKey(0), shape = (batch_size, nb_nodes, hidden_dim))#jnp.zeros((batch_size, nb_nodes, hidden_dim))
edge_fts = random.uniform(random.PRNGKey(0), shape = (batch_size, nb_nodes, nb_nodes, hidden_dim))#jnp.zeros((batch_size, nb_nodes, nb_nodes, hidden_dim))
graph_fts = random.uniform(random.PRNGKey(0), shape = (batch_size, hidden_dim))#jnp.zeros((batch_size, hidden_dim))
adj_mat = random.randint(random.PRNGKey(0), (batch_size, nb_nodes, nb_nodes), 0, 2, dtype='uint8')
hiddens = jnp.zeros((batch_size, nb_nodes, hidden_dim))

rng = jax.random.PRNGKey(42)
data = (node_fts, edge_fts, graph_fts, adj_mat, hiddens)
params = loss_fn_t.init(rng, data)


def loss(params, data):
  rng = jax.random.PRNGKey(42)
  return loss_fn_t.apply(params, rng, data)

def update(params, data, opt_state):
  step_size = 0.1
  rng = jax.random.PRNGKey(42)
  l, grads = jax.value_and_grad(loss)(params, data)
  grads, opt_state = opt.update(grads, opt_state)
  params = optax.apply_updates(params, grads)
  return l, params, opt_state

opt = optax.adam(1e-3)
opt_state = opt.init(params)


train_loss, params, opt_state = update(params, data, opt_state)


In [4]:
params

{'gatv2_aggr/linear': {'b': DeviceArray([-0.00999989,  0.        ,  0.        ,  0.        ,
                0.        ,  0.        ,  0.        ,  0.        ,
               -0.0099999 ,  0.        ], dtype=float32),
  'w': DeviceArray([[ 0.3359469 , -0.36728093,  0.13166161, -0.3212381 ,
                 0.49965456,  0.60795635, -0.6612269 ,  0.15789443,
                 0.23964234, -0.15423125],
               [-0.1603975 , -0.5239641 , -0.7315263 , -0.3995566 ,
                -0.509449  ,  0.4193997 ,  0.30625954,  0.14001477,
                 0.26148444,  0.02671466],
               [-0.12918024,  0.2374276 , -0.48345175,  0.28607336,
                -0.27448115, -0.29502004, -0.34856585, -0.23569146,
                 0.19813031,  0.6340319 ],
               [ 0.19778636,  0.49217984, -0.03938646,  0.14426231,
                 0.21487533, -0.24773812, -0.7956939 ,  0.00333428,
                -0.49984488,  0.22873653],
               [ 0.6027828 , -0.63268894, -0.25770262,  0.683